# Merge Large Language Models with mergekit
> 🗣️ [Large Language Model Course](https://github.com/mlabonne/llm-course)

❤️ Created by [@maximelabonne](https://twitter.com/maximelabonne).

Model merging only requires a lot of RAM. With a free Google Colab account, you should be able to run it using a T4 GPU (VRAM offloading).

Examples of merge configurations:

### TIES-Merging

```yaml
models:
  - model: mistralai/Mistral-7B-v0.1
    # no parameters necessary for base model
  - model: OpenPipe/mistral-ft-optimized-1218
    parameters:
      density: 0.5
      weight: 0.5
  - model: mlabonne/NeuralHermes-2.5-Mistral-7B
    parameters:
      density: 0.5
      weight: 0.3
merge_method: ties
base_model: mistralai/Mistral-7B-v0.1
parameters:
  normalize: true
dtype: float16
```

You can find the final model on the Hugging Face Hub at [mlabonne/NeuralPipe-7B-ties](https://huggingface.co/mlabonne/NeuralPipe-7B-ties).

### SLERP

```yaml
slices:
  - sources:
      - model: OpenPipe/mistral-ft-optimized-1218
        layer_range: [0, 32]
      - model: mlabonne/NeuralHermes-2.5-Mistral-7B
        layer_range: [0, 32]
merge_method: slerp
base_model: OpenPipe/mistral-ft-optimized-1218
parameters:
  t:
    - filter: self_attn
      value: [0, 0.5, 0.3, 0.7, 1]
    - filter: mlp
      value: [1, 0.5, 0.7, 0.3, 0]
    - value: 0.5
dtype: bfloat16
```

You can find the final model on the Hugging Face Hub at [mlabonne/NeuralPipe-7B-slerp](https://huggingface.co/mlabonne/NeuralPipe-7B-slerp).

### Passthrough

```yaml
slices:
  - sources:
    - model: OpenPipe/mistral-ft-optimized-1218
      layer_range: [0, 32]
  - sources:
    - model: mlabonne/NeuralHermes-2.5-Mistral-7B
      layer_range: [24, 32]
merge_method: passthrough
dtype: bfloat16
```

You can find the final model on the Hugging Face Hub at [mlabonne/NeuralPipe-9B-merged](https://huggingface.co/mlabonne/NeuralPipe-9B-merged).

In [ ]:
!git clone https://github.com/cg123/mergekit.git
!cd mergekit && pip install -q -e .

Cloning into 'mergekit'...
remote: Enumerating objects: 2356, done.
remote: Counting objects: 100% (799/799), done.
remote: Compressing objects: 100% (187/187), done.
remote: Total 2356 (delta 700), reused 617 (delta 612), pack-reused 1557 (from 1)
Receiving objects: 100% (2356/2356), 693.14 KiB | 6.66 MiB/s, done.
Resolving deltas: 100% (1635/1635), done.
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.9/330.9 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 94.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import yaml

MODEL_NAME = "QAIMath-Qwen2.5-7B-TIES"
yaml_config = """
models:
  - model: Qwen/Qwen2.5-Math-7B
    parameters:
      density: 0.5
      weight: 0.4
  - model: Qwen/Qwen2.5-Math-7B-Instruct
    parameters:
      density: 0.5
      weight: 0.3
merge_method: ties
base_model: Qwen/Qwen2.5-7B
parameters:
  normalize: true
dtype: bfloat16
"""

with open('config.yaml', 'w', encoding="utf-8") as f:
    f.write(yaml_config)


In [ ]:
# Merge models
!mergekit-yaml config.yaml merge --copy-tokenizer --allow-crimes --out-shard-size 1B --lazy-unpickle

Se han truncado las últimas 5000 líneas del flujo de salida.

model-00002-of-00004.safetensors:  14% 545M/3.86G [00:13<01:19, 42.0MB/s]



model-00004-of-00004.safetensors:  15% 545M/3.56G [00:13<01:11, 41.9MB/s]


model-00003-of-00004.safetensors:  14% 556M/3.86G [00:13<01:17, 42.5MB/s]






model-00001-of-00004.safetensors:  14% 556M/3.95G [00:13<01:19, 42.5MB/s]

model-00002-of-00004.safetensors:  14% 556M/3.86G [00:13<01:18, 42.1MB/s]



model-00004-of-00004.safetensors:  16% 556M/3.56G [00:13<01:10, 42.3MB/s]


model-00003-of-00004.safetensors:  15% 566M/3.86G [00:13<01:18, 42.1MB/s]






model-00001-of-00004.safetensors:  14% 566M/3.95G [00:13<01:19, 42.7MB/s]

model-00002-of-00004.safetensors:  15% 566M/3.86G [00:13<01:18, 42.2MB/s]



model-00004-of-00004.safetensors:  16% 566M/3.56G [00:13<01:10, 42.4MB/s]


model-00003-of-00004.safetensors:  15% 577M/3.86G [00:13<01:17, 42.4MB/s]






model-00001-of-00004.safetensors:  15% 577M/3.95G [00:13<01:19, 42.6MB/s]

model-00002-of

In [ ]:
!pip install -qU huggingface_hub

from huggingface_hub import ModelCard, ModelCardData
from jinja2 import Template

username = "adriszmar"

template_text = """
---
license: apache-2.0
tags:
- merge
- mergekit
- lazymergekit
{%- for model in models %}
- {{ model }}
{%- endfor %}
---

# {{ model_name }}

{{ model_name }} is a merge of the following models using [mergekit](https://github.com/cg123/mergekit):

{%- for model in models %}
* [{{ model }}](https://huggingface.co/{{ model }})
{%- endfor %}

## 🧩 Configuration

```yaml
{{- yaml_config -}}
```
"""

# Create a Jinja template object
jinja_template = Template(template_text.strip())

# Get list of models from config
data = yaml.safe_load(yaml_config)
if "models" in data:
    models = [data["models"][i]["model"] for i in range(len(data["models"])) if "parameters" in data["models"][i]]
elif "parameters" in data:
    models = [data["slices"][0]["sources"][i]["model"] for i in range(len(data["slices"][0]["sources"]))]
elif "slices" in data:
    models = [data["slices"][i]["sources"][0]["model"] for i in range(len(data["slices"]))]
else:
    raise Exception("No models or slices found in yaml config")

# Fill the template
content = jinja_template.render(
    model_name=MODEL_NAME,
    models=models,
    yaml_config=yaml_config,
    username=username,
)

# Save the model card
card = ModelCard(content)
card.save('merge/README.md')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.4/447.4 kB 6.5 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
from huggingface_hub import HfApi

username = "adriszmar"

# Defined in the secrets tab in Google Colab
api = HfApi(token=userdata.get("HF_TOKEN"))

api.create_repo(
    repo_id=f"{username}/{MODEL_NAME}",
    repo_type="model"
)
api.upload_folder(
    repo_id=f"{username}/{MODEL_NAME}",
    folder_path="merge",
)

model-00002-of-00016.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

model-00004-of-00016.safetensors:   0%|          | 0.00/932M [00:00<?, ?B/s]

model-00001-of-00016.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

model-00003-of-00016.safetensors:   0%|          | 0.00/932M [00:00<?, ?B/s]

model-00005-of-00016.safetensors:   0%|          | 0.00/932M [00:00<?, ?B/s]

...

model-00014-of-00016.safetensors:   0%|          | 0.00/932M [00:00<?, ?B/s]

model-00015-of-00016.safetensors:   0%|          | 0.00/932M [00:00<?, ?B/s]

model-00016-of-00016.safetensors:   0%|          | 0.00/932M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/adriszmar/QAIMath-Qwen2.5-7B-TIES/commit/c89bc166dbe2a31c1fceb40ea7acdd96c5620ff5', commit_message='Upload folder using huggingface_hub', commit_description='', oid='c89bc166dbe2a31c1fceb40ea7acdd96c5620ff5', pr_url=None, repo_url=RepoUrl('https://huggingface.co/adriszmar/QAIMath-Qwen2.5-7B-TIES', endpoint='https://huggingface.co', repo_type='model', repo_id='adriszmar/QAIMath-Qwen2.5-7B-TIES'), pr_revision=None, pr_num=None)